## IMPORT LIBRARIES AND OTHER CONFIG VARIABLES

In [40]:
import pandas as pd
import os
import boto3
import json
from dotenv import load_dotenv
from config import BUCKET_NAME, PR_PREFIX, POPULATION_PREFIX

In [41]:
# Load environment variables from .env file
load_dotenv()

True

In [42]:
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_DEFAULT_REGION = os.getenv("AWS_DEFAULT_REGION")

In [43]:
# Initialize S3 client with credentials
s3 = boto3.client(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_DEFAULT_REGION
)

## SECTION 0 - LOAD DATA FROM S3

In [44]:
# Load part 1 data into df
part1_data = s3.get_object(Bucket=BUCKET_NAME, Key=f"{PR_PREFIX}")
part1_df = pd.read_csv(part1_data['Body'], sep='\t')

In [45]:
display(part1_df.head())

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.6,NaN
1,PRS30006011,1995,Q02,2.1,NaN
2,PRS30006011,1995,Q03,0.9,NaN
3,PRS30006011,1995,Q04,0.1,NaN
4,PRS30006011,1995,Q05,1.4,NaN


In [46]:
# Load part 2 data into df
api_population = s3.get_object(Bucket=BUCKET_NAME, Key=POPULATION_PREFIX)
population_json = json.load(api_population["Body"])
part2_df = pd.json_normalize(population_json['data'])

In [47]:
display(part2_df.head(5))

,ID Nation,Nation,ID Year,Year,Population,Slug Nation
0,01000US,United States,2023,2023,332387540,united-states
1,01000US,United States,2022,2022,331097593,united-states
2,01000US,United States,2021,2021,329725481,united-states
3,01000US,United States,2020,2020,326569308,united-states
4,01000US,United States,2019,2019,324697795,united-states


## SECTION 1 - CALCULATING MEAN AND STD ACROSS YEARS 2013-2018

In [48]:
part2_df.dtypes

ID Nation      object
Nation         object
ID Year         int64
Year           object
Population      int64
Slug Nation    object
dtype: object

In [49]:
part2_df['Year'] = part2_df['Year'].astype(int)

In [50]:
df_pop_2013_2018 = part2_df[(part2_df['Year'] >= 2013) & (part2_df['Year'] <= 2018)]

In [51]:
mean_pop = round(df_pop_2013_2018['Population'].mean(),2)
std_pop = round(df_pop_2013_2018['Population'].std(),2)

# round to 2 decimal places
print("Mean Population (2013–2018):", mean_pop)
print("Standard Deviation:", std_pop)

Mean Population (2013–2018): 317437383.0
Standard Deviation: 4257089.54


## SECTION 2 - BEST YEAR BY SERIES_ID

In [34]:
part1_df.columns = part1_df.columns.str.strip()

In [35]:
display(part1_df.head())

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.6,NaN
1,PRS30006011,1995,Q02,2.1,NaN
2,PRS30006011,1995,Q03,0.9,NaN
3,PRS30006011,1995,Q04,0.1,NaN
4,PRS30006011,1995,Q05,1.4,NaN


In [36]:
part1_df.dtypes

series_id          object
year                int64
period             object
value             float64
footnote_codes     object
dtype: object

In [37]:
# Group and aggregate
grouped = part1_df.groupby(['series_id', 'year'])['value'].sum().reset_index()
best_years = grouped.loc[grouped.groupby('series_id')['value'].idxmax()].reset_index(drop=True)


display(best_years.head(20))


,series_id,year,value
0,PRS30006011,2022,20.500
1,PRS30006012,2022,17.100
2,PRS30006013,1998,705.895
3,PRS30006021,2010,17.700
4,PRS30006022,2010,12.400
5,PRS30006023,2014,503.216
6,PRS30006031,2022,20.500
7,PRS30006032,2021,17.100
8,PRS30006033,1998,702.672
9,PRS30006061,2022,37.000


## SECTION 3 - JOIN

In [39]:
# Filter for series_id and period
part1_df['series_id'] = part1_df['series_id'].str.strip()
part1_df['period'] = part1_df['period'].str.strip()
filtered = part1_df[(part1_df['series_id'] == 'PRS30006032') & (part1_df['period'] == 'Q01')]

# Prepare population data
pop_lookup = part2_df[['Year', 'Population']].drop_duplicates()
pop_lookup['Population'] = pd.to_numeric(pop_lookup['Population'], errors='coerce').astype('Int64')
pop_lookup.columns = ['year', 'Population']

# Merge
final = pd.merge(filtered, pop_lookup, on='year', how='right')
final_result = final[['series_id', 'year', 'period', 'value', 'Population']]
display(final_result)


,series_id,year,period,value,Population
0,PRS30006032,2023,Q01,0.3,332387540
1,PRS30006032,2022,Q01,5.3,331097593
2,PRS30006032,2021,Q01,0.7,329725481
3,PRS30006032,2020,Q01,-7.0,326569308
4,PRS30006032,2019,Q01,-1.6,324697795
5,PRS30006032,2018,Q01,0.5,322903030
6,PRS30006032,2017,Q01,0.9,321004407
7,PRS30006032,2016,Q01,-1.4,318558162
8,PRS30006032,2015,Q01,-1.7,316515021
9,PRS30006032,2014,Q01,-0.1,314107084
